In [1]:
from run_recorded_actions import run_action_list_on_emulator_and_save_video, run_recorded_actions_on_emulator_and_save_video


In [7]:
import pandas as pd
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from einops import rearrange
import requests
import io
import json
from functools import reduce
from tqdm import tqdm
from scipy.ndimage import gaussian_filter
import mediapy as media
import numpy as np

In [3]:
!ls -lth session_3e4969a1/stats_copy

total 452M
-rw-rw-r-- 1 pwhiddy pwhiddy 681K Oct  2 21:53 Visualizations_over_time.ipynb
-rw-rw-r-- 1 pwhiddy pwhiddy  15K Oct  2 20:54 Untitled.ipynb
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_eb35f1c2.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_eb21d3ff.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_e580ec6a.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_e501da75.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_e0102e40.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_c7ef6067.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_c167173a.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_b3720eb4.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_88f5a17a.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_796da009.csv.gz
-rw-rw-r-- 1 pwhiddy pwhiddy  29M Oct  2 20:41 agent_stats_50fc762b.csv.gz
-rw-rw-r-- 1 pwhiddy pwh

In [3]:
run_dir = Path("session_3e4969a1/stats_copy")

In [11]:
def path_to_runs(pth):
    tdf = pd.read_csv(pth, compression='gzip')
    print(len(tdf))
    return np.array_split(tdf[tdf['map'] != 'map'], 519)

In [12]:
all_runs = [path_to_runs(pth) for pth in tqdm(run_dir.glob('agent_stats_e*.gz'))] #tqdm(run_dir.glob('agent_stats_*.gz'))]
all_runs = list(map(list, zip(*all_runs)))

0it [00:00, ?it/s]/tmp/ipykernel_238068/103370422.py:2: DtypeWarning: Columns (1,2,3,4,5,6,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf = pd.read_csv(pth, compression='gzip')


4260359


1it [00:03,  3.53s/it]/tmp/ipykernel_238068/103370422.py:2: DtypeWarning: Columns (1,2,3,4,5,6,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf = pd.read_csv(pth, compression='gzip')


4260359


2it [00:06,  3.44s/it]/tmp/ipykernel_238068/103370422.py:2: DtypeWarning: Columns (1,2,3,4,5,6,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf = pd.read_csv(pth, compression='gzip')


...

3it [00:10,  3.38s/it]/tmp/ipykernel_238068/103370422.py:2: DtypeWarning: Columns (1,2,3,4,5,6,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf = pd.read_csv(pth, compression='gzip')


4260359


4it [00:13,  3.40s/it]/tmp/ipykernel_238068/103370422.py:2: DtypeWarning: Columns (1,2,3,4,5,6,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf = pd.read_csv(pth, compression='gzip')


4260359


5it [00:16,  3.39s/it]


In [15]:
len(all_runs[0][0])

8209

In [16]:
all_map_ids = set()
outer_indices = set()
for idx,outer in tqdm(enumerate(all_runs)):
    for inner in outer:
        for val in inner["map"].unique():
            if val == "3":
                outer_indices.add(idx)
            all_map_ids.add(val)

519it [00:00, 1570.95it/s]


In [17]:
outer_indices

{490, 492, 493, 494, 495}

In [22]:
inner_indices = set()
for idx, inner in enumerate(all_runs[490]):
    for val in inner["map"].unique():
        if val == "3":
            inner_indices.add(idx)


In [23]:
inner_indices

{0}

In [26]:
all_runs[490][0]

,Unnamed: 0,step,x,y,map,last_action,pcount,levels,ptypes,hp,coord_count,deaths,badge,event,healr
4022313,7743.0,7743,38,23,61,3,2,"[15, 11, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",1.0,1393,2,1,9,3.2315593711352104
4022314,7744.0,7744,38,23,61,7,2,"[15, 11, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",1.0,1393,2,1,9,3.2315593711352104
4022315,7745.0,7745,38,23,61,3,2,"[15, 11, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",1.0,1393,2,1,9,3.2315593711352104
4022316,7746.0,7746,38,23,61,7,2,"[15, 11, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",1.0,1393,2,1,9,3.2315593711352104
4022317,7747.0,7747,38,23,61,3,2,"[15, 11, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",1.0,1393,2,1,9,3.2315593711352104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030516,15946.0,15946,21,17,60,4,2,"[18, 12, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",1.0,596,3,1,13,3.6470543195248957
4030517,15947.0,15947,21,17,60,6,2,"[18, 12, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",1.0,596,3,1,13,3.6470543195248957
4030518,15948.0,15948,20,17,60,1,2,"[18, 12, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",1.0,596,3,1,13,3.6470543195248957
4030519,15949.0,15949,20,17,60,4,2,"[18, 12, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",1.0,596,3,1,13,3.6470543195248957


In [8]:
golden_goose = pd.read_csv("session_3e4969a1/stats_copy/agent_stats_eb35f1c2.csv.gz")

/tmp/ipykernel_241905/272524482.py:1: DtypeWarning: Columns (1,2,3,4,5,6,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  golden_goose = pd.read_csv("session_3e4969a1/stats_copy/agent_stats_eb35f1c2.csv.gz")


In [9]:
# walk down list until celadon is found, then walk back to beginning of game
celadon_idx = len(golden_goose) - 1
while golden_goose.iloc[celadon_idx]["map"] != "3":
    celadon_idx -= 1
    print(f"\r{celadon_idx}", end="")
while golden_goose.iloc[celadon_idx]["step"] != "0":
    celadon_idx -= 1
    print(f"\r{celadon_idx}", end="")
golden_range = (celadon_idx,celadon_idx+16383)


4030956

In [18]:
golden_goose[golden_range[0]-5+300:golden_range[0]+15+300]#golden_range[1]]["map"]

,Unnamed: 0,step,x,y,map,last_action,pcount,levels,ptypes,hp,coord_count,deaths,badge,event,healr
4031251,295.0,295,7,28,12,4,2,"[6, 0, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.45454545454545453,62,0,0,2,0.0
4031252,296.0,296,7,28,12,5,2,"[6, 3, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.6842105263157895,62,0,0,2,0.0
4031253,297.0,297,7,28,12,0,2,"[6, 3, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.6842105263157895,62,0,0,2,0.22966507177033496
4031254,298.0,298,7,28,12,4,2,"[6, 3, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.6842105263157895,62,0,0,2,0.22966507177033496
4031255,299.0,299,8,28,12,2,2,"[6, 3, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.6842105263157895,62,0,0,2,0.22966507177033496
4031256,300.0,300,8,27,12,3,2,"[6, 3, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.6842105263157895,62,0,0,2,0.22966507177033496
4031257,301.0,301,8,26,12,3,2,"[6, 3, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.6842105263157895,62,0,0,2,0.22966507177033496
4031258,302.0,302,9,26,12,2,2,"[6, 3, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.6842105263157895,62,0,0,2,0.22966507177033496
4031259,303.0,303,10,26,12,2,2,"[6, 3, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.6842105263157895,63,0,0,2,0.22966507177033496
4031260,304.0,304,11,26,12,2,2,"[6, 3, 0, 0, 0, 0]","[177, 36, 255, 0, 0, 0]",0.6842105263157895,64,0,0,2,0.22966507177033496


In [12]:
#golden_goose[16384*10:16384*10+30]#golden_range[1]]["map"]

In [14]:
golden_actions = [int(a) for a in list(golden_goose[golden_range[0]:golden_range[1]]["last_action"])]

In [17]:
golden_actions[:15]

[0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 2, 1, 1, 1, 1]

In [15]:
run_action_list_on_emulator_and_save_video(golden_actions[:500])

134067   pyboy.plugins.window_headless  WARNING  This window type does not support frame-limiting. `pyboy.set_emulation_speed(...)` will have no effect, as it's always running at full speed.
100%|████████████████████████████████████████████████████████████████████████████| 500/500 [00:18<00:00, 26.50it/s]


In [ ]:
#run_recorded_actions_on_emulator_and_save_video("5586ece3", "28084aad", 1)

In [2]:
run_recorded_actions_on_emulator_and_save_video("3e4969a1", "eb35f1c2", 490 // 2)

7180     pyboy.plugins.window_headless  WARNING  This window type does not support frame-limiting. `pyboy.set_emulation_speed(...)` will have no effect, as it's always running at full speed.


267


 15%|███████████                                                              | 2471/16385 [01:32<08:43, 26.57it/s]


KeyboardInterrupt: 

In [6]:
490 // 2

245